# Pandas tutorial

Dataset characteristic:

- every folder contain: addresses.csv, addresses_people.csv, people.csv, people_publications.csv, publications.csv.
- every file contain few columns, named in first row
- every file not with many-to-many relations, contain temp_id column

Project:

- merge all files into single DataFrame
- change current temp_ids into new unique ids
- eliminate duplicates (eg. consider address with x percentage of similarity as one)
- add column “town” for people.csv data, extracted from “addresses” column in addresses.csv
- get missing lat/lng data for every town/address (eg. through google api)
- save DataFrame to single csv

## Variables

In [24]:
similarity_percentage = 20

data_directory="./data"
data_sub_folders = 134
data_load_step = 1

## Functions & Imports

In [25]:
import re
import pandas as pd
import os

from typing import List

def move_head(arr, index: int):
    arr = arr = [arr[index]] + arr[:index] + arr[index + 1:]

def data_filter(name: str, address: str):
    data = name.split(",") + address.split(",")
    filtered_data = []
    for word in data:
        cleaned_word = word.strip().lower()
        cleaned_word = re.sub(r'[^a-zA-Z\s]', ' ', cleaned_word)
        cleaned_word = re.sub(r'\s+', ' ', cleaned_word)
        if len(cleaned_word) > 2:
            filtered_data.append(cleaned_word.strip())
    return ",".join(filtered_data)
            
def similarity(data1: str, data2: str):
    arr1, arr2 = data1.split(","), data2.split(",")
    counter = 0
    total = max(len(arr1), len(arr2))
    for str1 in arr1:
        for str2 in arr2:
            if str1 in str2 or str2 in str1:
                counter += 1
    return counter / total * 100

def make_id_unique(df, value, column="temp_id"):
    df[column] = one_one_df[column].apply(lambda id: str(value) + "_" + str(id))
    return df

class GoodAddress:
    string: str
    uuid: str
    children_list: list
    def __init__(self, string, uuid) -> None:
        self.string = string
        self.uuid = uuid
        self.children_list = []

## Load data

In [27]:
data_files = {"addresses": [], "addresses_people": [], "people": [], "people_publications": [], "publications": []}

for i in range(0, data_sub_folders, data_load_step):
    data_files["addresses"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/ADDRESSES.csv")), i))
    data_files["addresses_people"].append(make_id_unique(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/ADDRESSES_PEOPLE.csv")), i, "address_uuid"), i, "person_uuid"))
    data_files["people"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PEOPLE.csv")), i))
    data_files["people_publications"].append(make_id_unique(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PEOPLE_PUBLICATIONS.csv")), i, "person_uuid"), i, "publication_uuid"))
    data_files["publications"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PUBLICATIONS.csv")), i))

address_df = pd.concat(data_files["addresses"], ignore_index=True)
addresses_people_df = pd.concat(data_files["addresses_people"], ignore_index=True)
people_df = pd.concat(data_files["people"], ignore_index=True)
people_publications_df = pd.concat(data_files["people_publications"], ignore_index=True)
publications_df = pd.concat(data_files["publications"], ignore_index=True)

address_df.rename(columns={"temp_id": "address_uuid"}, inplace=True)
people_df.rename(columns={"temp_id": "person_uuid"}, inplace=True)
publications_df.rename(columns={"temp_id": "publication_uuid"}, inplace=True)

address_df

,address_uuid,address,countries_scope,lat,lon,name,phone,url
0,0_2,"sogn og fjordane university collegesogndal, no...",NO,NaN,NaN,faculty of teacher education and sport,NaN,NaN
1,0_3,"university of bergenbergen, norway",NO,NaN,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN
2,0_4,"haukeland university hospitalbergen, norway",NO,NaN,NaN,department of pathology,NaN,NaN
3,0_4,"haukeland university hospitalbergen, norway",NO,NaN,NaN,department of pathology,NaN,NaN
4,0_5,"university of bergenbergen, norway",NO,NaN,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN
...,...,...,...,...,...,...,...,...
203334,133_1464,division of environmental medicinenorwegian in...,NO,NaN,NaN,department of air pollution and noise,NaN,NaN
203335,133_1465,division of environmental medicinenorwegian in...,NO,NaN,NaN,department of air pollution and noise,NaN,NaN
203336,133_1466,division of environmental medicinenorwegian in...,NO,NaN,NaN,department of air pollution and noise,NaN,NaN
203337,133_1467,division of environmental medicinenorwegian in...,NO,NaN,NaN,department of air pollution and noise,NaN,NaN


## Prepare good addresses indexes

In [28]:
%%time

address_df["full_address"] = address_df.apply(lambda x: data_filter(str(x["name"]), str(x["address"])), axis=1)

good_address_list: List[GoodAddress] = []

for address_index, address in address_df.iterrows():
    flag = True
    for good_address_index, ga in enumerate(good_address_list):
        if similarity(ga.string, address["full_address"]) > similarity_percentage:
            flag = False
            ga.children_list.append(address["address_uuid"])
            move_head(good_address_list, good_address_index)
            break
    if flag:    
        good_address_list.append(GoodAddress(address["full_address"], address["address_uuid"]))

address_df = address_df.drop(["full_address"], axis=1)

print(f"{len(good_address_list)} good addresses found")

217 good addresses found
CPU times: user 14.9 s, sys: 7.96 ms, total: 15 s
Wall time: 15 s


## Update ADDRESSES_PEOPLE relation 

In [29]:
replace_map = {to_replace_address_id: good_address.uuid for good_address in good_address_list for to_replace_address_id in good_address.children_list}
addresses_people_df["address_uuid"] = addresses_people_df["address_uuid"].map(replace_map).fillna(addresses_people_df["address_uuid"])
unique_address_count = addresses_people_df['address_uuid'].nunique()
print(unique_address_count <= len(good_address_list))

True


# Merge

In [38]:
merged_1 = pd.merge(people_df, addresses_people_df, on='person_uuid', how="inner")
merged_2 = pd.merge(merged_1, address_df, on="address_uuid", how="inner")
merged_3 = pd.merge(merged_2, people_publications_df, on="person_uuid", how="inner")
one_df = pd.merge(merged_3, publications_df, on="publication_uuid", how="inner")

len(one_df)

204431

## Unify ID

In [41]:
one_df['id'] = one_df['address_uuid'].astype(str) + "_" + one_df['person_uuid'].astype(str) + "_" + one_df['publication_uuid'].astype(str)
one_df = one_df[['id'] + [col for col in one_df.columns if col != 'id']]
one_df.drop(columns=['address_uuid', 'person_uuid', "publication_uuid"], inplace=True)


,id,lastname,firstname,countries_scope_x,email,town,role,url_x,address,countries_scope_y,...,lon,name,phone,url_y,title,pubmed_id,journal,year,url,abstract
0,0_2_0_1_0_1,Andersen,Lars B,NO,NaN,sogndal,NaN,NaN,"sogn og fjordane university collegesogndal, no...",NO,...,NaN,faculty of teacher education and sport,NaN,NaN,Cardiorespiratory fitness cut points to avoid ...,27670254,British journal of sports medicine. Volume: 50...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27670254/,Poor cardiorespiratory fitness is associated w...
1,0_3_0_2_0_2,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"university of bergenbergen, norway",NO,...,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
2,0_2_0_2_0_2,Knutsvik,Gøril,NO,NaN,bergen,NaN,NaN,"sogn og fjordane university collegesogndal, no...",NO,...,NaN,faculty of teacher education and sport,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
3,0_2_0_3_0_2,Collett,Karin,NO,NaN,bergen,NaN,NaN,"sogn og fjordane university collegesogndal, no...",NO,...,NaN,faculty of teacher education and sport,NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
4,0_3_0_4_0_2,Arnes,Jarle,NO,NaN,bergen,NaN,NaN,"university of bergenbergen, norway",NO,...,NaN,"centre for cancer biomarkers, ccbio, departmen...",NaN,NaN,QSOX1 expression is associated with aggressive...,27562495,Modern pathology : an official journal of the ...,2016.0,http://www.ncbi.nlm.nih.gov/pubmed/27562495/,The biological role of quiescin sulfhydryl oxi...
